In [21]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession
    .builder
    .appName("Capítulo 4")
    .getOrCreate()


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@292dba65


In [3]:
val csvFile = "../data/departuredelays.csv"
val df = spark.read.format("csv")
    .option("inferSchema","true")
    .option("header","true")
    .load(csvFile)
df.createOrReplaceTempView("us_delay_flights_tbl")

csvFile: String = ../data/departuredelays.csv
df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]


In [5]:
spark.sql("""SELECT distance, origin, destination FROM us_delay_flights_tbl
WHERE distance > 1000 ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [8]:
spark.sql("""SELECT date, delay, origin, destination FROM us_delay_flights_tbl
WHERE delay > 120 AND origin = 'SFO' and destination = 'ORD' ORDER BY delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [9]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


In [12]:
// Función para convertir 
def toDateFormatUDF(dStr:String) : String  = {
  return s"${dStr(0)}${dStr(1)}${'/'}${dStr(2)}${dStr(3)}${' '}${dStr(4)}${dStr(5)}${':'}${dStr(6)}${dStr(7)}"
}

// test  it
toDateFormatUDF("02190925")

toDateFormatUDF: (dStr: String)String
res7: String = 02/19 09:25


In [13]:
// Registrar función
spark.udf.register("toDateFormatUDF", toDateFormatUDF(_:String):String)

res8: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$4096/0x0000000801b69fc0@775d6afb,StringType,List(Some(class[value[0]: string])),Some(class[value[0]: string]),Some(toDateFormatUDF),true,true)


In [19]:
//spark.sql("""SELECT toDateFormatUDF(date) as date_t, delay, origin, destination FROM us_delay_flights_tbl
//WHERE delay > 120 AND origin = 'SFO' AND destination = 'ORD' ORDER BY delay DESC""").show(4)

//df.selectExpr("toDateFormatUDF(date) as data_format").show(10, false)

In [20]:
spark.sql("""SELECT delay, origin, destination,
    CASE
        WHEN delay > 360 THEN 'Very Long Delays'
        WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
        WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
        WHEN delay > 0 AND delay < 60 THEN 'Tolerable Delays'
        WHEN delay = 0 THEN 'No Delays'
        ELSE 'Early'
    END AS Flight_Delays
    FROM us_delay_flights_tbl
    ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



**GlobalTempView vs TempView**

Una vista Temporal Global se puede acceder desde cualquier SparkSession, mientras que si no es Global solo se puede acceder
desde la SparkSession desde donde se ha creado.

**Leer los AVRO, Parquet, JSON y CSV escritos en el cap3**

In [ ]:
val avroDF = spark.read.format("avro").load("../data/saved/fileavro/*")
val parquetDF = spark.read.format("parquet").load("../data/saved/fileparquet/*")
val jsonDF = spark.read.format("json").option("inferSchema","true").load("../data/saved/filejson/*")
val csvDF = spark.read.format("csv").option("header","true").option("inferSchema","true").load("../data/saved/filecsv/*")